In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("parsl@parsl.csv")
data = data.to_numpy()

data_process = pd.read_csv("process_parsl.csv")
data_process = data_process.to_numpy()

In [ ]:
# remove non-test build
print(len(data))
test_build_data = []

for row in data:
    if row[9] == True:
        test_build_data.append(row)

print(len(test_build_data))

In [ ]:
# combine same build
unique_build = []

prev_build = test_build_data[0][0]
prev_commit = test_build_data[0][3]
test_string = str(test_build_data[0][18])

for i in range(1, len(test_build_data)+1):
    if i == len(test_build_data):
        unique_build.append([prev_build, prev_commit, test_string])
        break
    
    if test_build_data[i][0] == prev_build:
        test_string = test_string + "#" + str(test_build_data[i][18])
    else:
        unique_build.append([prev_build, prev_commit, test_string])
        prev_build = test_build_data[i][0]
        prev_commit = test_build_data[i][3]
        test_string = str(test_build_data[i][18])

# clear same tests
for i in range(len(unique_build)):
    temp_tests = []
    
    for item in unique_build[i][2].split("#"):
        if item !="nan":
            if item not in temp_tests:
                temp_tests.append(item)
    
    write_tests = ""
    for t in temp_tests:
        if write_tests == "":
            write_tests = write_tests + t
        else:
            write_tests = write_tests + "#" + t
        
    unique_build[i][2] = write_tests

print(unique_build)

In [ ]:
# combine two data set to [build ID, commit, file_change, tests] - Commit Guru + Travis CI
combined_data = []

count_overlap = 0
count_no_overlap = 0
for i in range(len(unique_build)):
    findSame = False
    
    for j in range(len(data_process)):
        if unique_build[i][1] == data_process[j][0]:
            findSame = True
            new_row = [unique_build[i][0], unique_build[i][1], data_process[j][17], unique_build[i][2]]
            combined_data.append(new_row)
            break
    
    if findSame:
        count_overlap += 1
    else:
        count_no_overlap += 1

print("overlap: " + str(count_overlap) + " " + str(count_overlap/len(unique_build)))
print("non-overlap: " + str(count_no_overlap) + " " + str(count_no_overlap/len(unique_build)))

In [ ]:
# scrap file data from Github
import csv
import requests
from bs4 import BeautifulSoup as bs
import random
import time

with open("parsl_combined.csv", "w", newline='', encoding="utf-8") as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerow(['build ID', 'commit', 'time', 'file change', 'tests'])
    
    for row in unique_build:
        build_id = row[0]
        commit_id = row[1]
        test_info = row[2]
        
        url = "https://github.com/Parsl/parsl/commit/" + commit_id
        print(url)
        
        r = requests.get(url)
        soup = bs(r.content, "html.parser")
        
        myDiv = soup.findAll('div', {'class': 'file-info flex-auto min-width-0 mb-md-0 mb-2'})
        
        temp_file_list = []
        for div in myDiv:
            temp_file_list.append(div.find('a').text)
            
        timeDiv = soup.find('relative-time')
        if timeDiv is not None:
            commit_time = timeDiv.get('datetime')
            print(commit_time)
        else:
            commit_time = ""
        
        output_file_string = ""
        for f in temp_file_list:
            if output_file_string == "":
                output_file_string = output_file_string + f
            else:
                output_file_string = output_file_string + "#" + f
        print(output_file_string)
        
        csv_writer.writerow([build_id, commit_id, commit_time, output_file_string, test_info])
        
        rand_int = random.randint(1, 4)
        time.sleep(rand_int)

In [ ]:
combined_data = pd.read_csv("parsl_combined.csv")
combined_data = combined_data.to_numpy()

failed_data = []
for row in combined_data:
    if str(row[4]) != "nan":
        failed_data.append(row)

print(len(combined_data))
print(len(failed_data))

In [ ]:
# train test split
tot_len = len(failed_data)
testing_len = int(tot_len * 0.7)
testing_data = failed_data[testing_len:]
print(testing_len)

split_build_id = failed_data[testing_len][0]

training_data = []

for row in combined_data:
    if row[0] == split_build_id:
        break
    
    training_data.append(row)

print(len(training_data))
print(len(testing_data))

In [ ]:
# create File-Commit-Timestep Table
file_list = []
for row in training_data:
    temp_file = row[3]
    
    for f in temp_file.split("#"):
        if f not in file_list:
            file_list.append(f)

file_commit_table = []

for row in training_data:
    cur_list = []
    cur_list.append(row[1])
    
    for file in file_list:
        if file in row[3].split('#'):
            cur_list.append(1)
        else:
            cur_list.append(0)
    
    cur_list.append(row[2])
    
    file_commit_table.append(cur_list)

print(file_commit_table)

In [ ]:
# collect test cases, build test-file Table
test_list = []
for row in failed_data:
    temp_test = row[4]
    
    for t in temp_test.split("#"):
        if t not in test_list:
            test_list.append(t)

test_file_table = [[0 for i in range(len(test_list))] for j in range(len(file_list))]
for row in training_data:
    if str(row[4]) != "nan":
        for i in range(len(file_list)):
            if file_list[i] in row[3].split("#"):
                for j in range(len(test_list)):
                    if test_list[j] in row[4].split("#"):
                        test_file_table[i][j] += 1

print(test_file_table)           

In [ ]:
# timestamp for each commit
import datetime

list_timestamp = []
list_commit = []
for row in file_commit_table:
    list_timestamp.append(row[-1])
    list_commit.append(row[1])

original_date = datetime.datetime(int(list_timestamp[0][0:4]), int(list_timestamp[0][5:7]), int(list_timestamp[0][8:10]), 
                                  int(list_timestamp[0][11:13]), int(list_timestamp[0][14:16]), int(list_timestamp[0][17:19]))

norm_timestamp = []
for item in list_timestamp:
    temp_date = datetime.datetime(int(item[0:4]), int(item[5:7]), int(item[8:10]),
                                  int(item[11:13]), int(item[14:16]), int(item[17:19]))
    
    diff = temp_date - original_date
    diff_sec = diff.total_seconds()
    score = diff_sec * 1
    norm_timestamp.append(score+5)

file_score = {}
for f in file_list:
    file_score.update({f: 0})

for i in range(len(training_data)):
    tc = norm_timestamp[i]/max(norm_timestamp)
    
    for f in training_data[i][3].split("#"):
        file_score[f] += tc

for key in file_score:
    print(key + ": " + str(file_score[key]))